In [1]:
# Import necessary libraries
import pandas as pd
from NLP_TFIDF import get_recommendations
from User_based import recommend_items
# from collaborative_recommendation import load_ratings_data, compute_user_item_matrix, compute_item_similarity, get_collaborative_recommendations

df_books = pd.read_csv('merged_books.csv')
df_ratings = pd.read_csv('Data/Ratings.csv')

# Filter the ratings dataset to include only the books present in merged_books.csv
filtered_ratings = df_ratings[df_ratings['ISBN'].isin(df_books['ISBN'])]

# Save the filtered dataset to a new CSV file
filtered_ratings.to_csv('filtered_ratings.csv', index=False)


df_books = pd.read_csv('merged_books.csv')
print("--------")
# print(df_books.head())
print("--------")

# print(get_recommendations("Destiny"))


recommended_content_based = get_recommendations("Destiny")
recommended_user_based = recommend_items(276746, 10)
 
#get the title of the recommend_items books
def get_title(book_id):
    return df_books[df_books['ISBN'] == book_id]['Book-Title'].values[0]

# print(recommended_)
for i in recommended_user_based:
    print(get_title(i))

for i in recommended_content_based:
    print(i)
    



# # Load data
# path = './Data/'
# df_books = load_books_data(path)
# df_ratings = load_ratings_data(path)

# # Compute TF-IDF matrix for content-based filtering
# tfidf_matrix = compute_tfidf_matrix(df_books)

# # Compute user-item matrix and item similarity for collaborative filtering
# user_item_matrix = compute_user_item_matrix(df_ratings)
# item_similarity_df = compute_item_similarity(user_item_matrix)

# # Combined Recommendation Function
# def get_combined_recommendations(user_id, title, k=5):
#     content_recommendations = get_content_based_recommendations(title, df_books, tfidf_matrix)
#     collaborative_recommendations = get_collaborative_recommendations(user_id, user_item_matrix, item_similarity_df, k)
#     return {
#         'Content-Based Recommendations': content_recommendations,
#         'Collaborative Recommendations': collaborative_recommendations
#     }

# # Example usage
# user_id = 10
# title = "The Kitchen God's Wife"
# recommendations = get_combined_recommendations(user_id, title)
# print(f"Recommendations for user {user_id} based on the book '{title}':")
# print("Content-Based Recommendations:", recommendations['Content-Based Recommendations'])
# print("Collaborative Recommendations:", recommendations['Collaborative Recommendations'])

Cosine similarity matrix loaded from 'cosine_similarity_matrix.npy'
6803              Sense of Evil
1824                     Chains
6988                   Ricochet
9839             One for Sorrow
5568     The Portrait of a Lady
1300                   The Heir
9635           Keeping the Moon
4905                       Lies
11209                Wide Awake
11135       The Collected Poems
Name: Book-Title, dtype: object
--------
--------
Bridget Jones: The Edge of Reason
Jaws
The Rainmaker
Split Second
The Beach
Saving Faith
The Deep End of the Ocean
The Catcher in the Rye
Spindle's End
The Associate
How to Get Rich
The Amateur Marriage
Gulliver's Travels
The Tale of the Flopsy Bunnies
In the Fall
The Tale of Tom Kitten
The Millionaire Next Door: The Surprising Secrets of America's Wealthy
Growth of the Soil
One Hundred Years of Solitude
Self


In [3]:
# Import necessary libraries
import pandas as pd
from NLP_TFIDF import get_recommendations
from User_based import recommend_items
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load datasets
df_books = pd.read_csv('merged_books.csv')
df_ratings = pd.read_csv('Data/Ratings.csv')

# Filter the ratings dataset to include only the books present in merged_books.csv
filtered_ratings = df_ratings[df_ratings['ISBN'].isin(df_books['ISBN'])]

# Save the filtered dataset to a new CSV file
filtered_ratings.to_csv('filtered_ratings.csv', index=False)

# Reload the books dataset
df_books = pd.read_csv('merged_books.csv')

# Compute TF-IDF matrix for content-based filtering
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df_books['description'].fillna(''))

# Function to get the title of the recommended books
def get_title(book_id):
    return df_books[df_books['ISBN'] == book_id]['Book-Title'].values[0]

# Function to get content-based relevance score
def get_content_relevance(book_id, title, df_books, tfidf_matrix):
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    idx = df_books[df_books['Book-Title'] == title].index[0]
    book_idx = df_books[df_books['ISBN'] == book_id].index[0]
    return cosine_sim[idx][book_idx]

# Combined Recommendation Function
def get_combined_recommendations(user_id, title, k=10, threshold=0.1):
    # Get user-based recommendations
    user_based_recommendations = recommend_items(user_id, k)
    for book in user_based_recommendations:
        print(get_title(book))
    
    print("--------+++++---------")
    # Filter user-based recommendations using content-based relevance
    combined_recommendations = []
    for book_id in user_based_recommendations:
        relevance = get_content_relevance(book_id, title, df_books, tfidf_matrix)
        if relevance >= threshold:
            combined_recommendations.append(get_title(book_id))
    
    # If fewer than k books pass the threshold, fill in with content-based recommendations
    if len(combined_recommendations) < k:
        content_based_recommendations = get_recommendations(title)
        for rec in content_based_recommendations:
            if len(combined_recommendations) >= k:
                break
            if rec not in combined_recommendations:
                combined_recommendations.append(rec)
                print(rec)
        print("done")
    
    return combined_recommendations

# Example usage
user_id = 276746
book_title = "Destiny"
recommendations = get_combined_recommendations(user_id, book_title, k=10, threshold=0.01)
print(f"Recommendations for user {user_id} based on the book '{book_title}':")
for rec in recommendations:
    print(rec)

Bridget Jones: The Edge of Reason
Jaws
The Rainmaker
Split Second
The Beach
Saving Faith
The Deep End of the Ocean
The Catcher in the Rye
Spindle's End
The Associate
--------+++++---------
How to Get Rich
The Amateur Marriage
Gulliver's Travels
The Tale of the Flopsy Bunnies
In the Fall
The Tale of Tom Kitten
done
Recommendations for user 276746 based on the book 'Destiny':
The Rainmaker
Saving Faith
The Deep End of the Ocean
The Associate
How to Get Rich
The Amateur Marriage
Gulliver's Travels
The Tale of the Flopsy Bunnies
In the Fall
The Tale of Tom Kitten
